In [ ]:
def upsample_block(x, in_channels, out_channels, scale=2):
    """
    Upsampling via transposed convolution (deconvolution).
    Doubles the spatial size for scale=2.
    """
    # ConvTranspose2d expects weight shape: [in_channels, out_channels, kH, kW]
    weight = torch.randn(in_channels, out_channels, 4, 4, device=x.device, requires_grad=True)
    
    # Perform deconvolution based upsampling
    x = F.conv_transpose2d(x, weight, stride=scale, padding=1)
    x = F.relu(x)
    return x, [weight]


In [ ]:
def feature_extraction(x, channels=64):

    # W1: convolution kernel of size 3x3x64
    # zero padding (padding=1) keeps output same size
    weight = torch.randn(channels, channels, 3, 3, device=x.device, requires_grad=True)
    bias = torch.zeros(channels, device=x.device, requires_grad=True)
    
    x = F.conv2d(x, weight, bias=bias, stride=1, padding=1)
    
    # ReLU (max(0, ·))
    x = F.relu(x)
    
    return x, [weight, bias]


In [ ]:
def codec_denoising(x, channels):
    
    w_down = torch.randn(channels, channels, 3, 3, device=x.device, requires_grad=True)
    w_up = torch.randn(channels, channels, 4, 4, device=x.device, requires_grad=True)

    h1 = F.relu(F.conv2d(x, w_down, stride=2, padding=1))
    h2 = F.conv_transpose2d(h1, w_up, stride=2, padding=1)

    out = x + h2  # local residual
    return out, [w_down, w_up]


In [ ]:
def reconstruction_block(x, channels):
    
    dilations = [1, 1, 2, 4]
    weights = []
    for d in dilations:
        w = torch.randn(channels, channels, 3, 3, device=x.device, requires_grad=True)
        x = F.relu(F.conv2d(x, w, padding=d, dilation=d))
        weights.append(w)
    return x, weights


In [ ]:
def final_projection(x, in_channels, out_channels=3):
    
    w = torch.randn(out_channels, in_channels, 1, 1, device=x.device, requires_grad=True)
    x = F.conv2d(x, w)
    return x, [w]


In [ ]:
def forward_pipeline(x, params, scale=2, base_channels=64):
    # Upsampling
    x = F.conv_transpose2d(x, params["w_up"], bias=params["b_up"], stride=scale, padding=1)
    x = F.relu(x)

    # Feature extraction
    f1 = F.relu(F.conv2d(x, params["w_feat"], bias=params["b_feat"], stride=1, padding=1))

    # Codec denoising
    h1 = F.relu(F.conv2d(f1, params["w_down"], bias=params["b_down"], stride=2, padding=1))
    h2 = F.conv_transpose2d(h1, params["w_up_codec"], bias=params["b_up_codec"], stride=2, padding=1)
    f2 = f1 + h2

    # Reconstruction (dilated convs)
    f3 = f2
    for i, d in enumerate([1, 1, 2, 4]):
        f3 = F.relu(F.conv2d(f3, params[f"w_rec_{i}"], bias=params[f"b_rec_{i}"], padding=d, dilation=d))

    f4 = f3 + f1
    out = F.conv2d(f4, params["w_final"], bias=params["b_final"])
    return out


In [ ]:
import torch
import torch.nn.functional as F

# ✅ STEP 1 — Initialize model parameters ONCE
def init_params(base_channels=64, scale=2, in_channels=3, out_channels=3):
    params = {}

    # small random weights
    params["w_up"] = torch.nn.Parameter(torch.randn(in_channels, base_channels, 4, 4) * 0.001)
    params["b_up"] = torch.nn.Parameter(torch.zeros(base_channels))

    params["w_feat"] = torch.nn.Parameter(torch.randn(base_channels, base_channels, 3, 3) * 0.001)
    params["b_feat"] = torch.nn.Parameter(torch.zeros(base_channels))

    params["w_down"] = torch.nn.Parameter(torch.randn(base_channels, base_channels, 3, 3) * 0.001)
    params["b_down"] = torch.nn.Parameter(torch.zeros(base_channels))
    params["w_up_codec"] = torch.nn.Parameter(torch.randn(base_channels, base_channels, 4, 4) * 0.001)
    params["b_up_codec"] = torch.nn.Parameter(torch.zeros(base_channels))

    for i, d in enumerate([1, 1, 2, 4]):
        params[f"w_rec_{i}"] = torch.nn.Parameter(torch.randn(base_channels, base_channels, 3, 3) * 0.001)
        params[f"b_rec_{i}"] = torch.nn.Parameter(torch.zeros(base_channels))

    params["w_final"] = torch.nn.Parameter(torch.randn(out_channels, base_channels, 1, 1) * 0.001)
    params["b_final"] = torch.nn.Parameter(torch.zeros(out_channels))

    return list(params.values()), params


In [ ]:
# Dataloader
train_data = SRDataset([
"GIVE YOUR DATASET PATH HERE"
], scale=2, patch_size=64, augment=True)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Initialize model parameters ONCE
params_list, params = init_params(base_channels=64, scale=2)

# Optimizer (start with Adam for stability)
optimizer = torch.optim.Adam(params_list, lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

# Training loop
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    total_loss = 0
    for i, (lr_imgs, hr_imgs) in enumerate(train_loader):
        optimizer.zero_grad()
        sr_imgs = forward_pipeline(lr_imgs, params, scale=2)
        loss = F.mse_loss(sr_imgs, hr_imgs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if i >= 1000:
            break

    scheduler.step()
    avg_loss = total_loss / min(1000, len(train_loader))
    print(f"Epoch {epoch:03d} | Loss: {avg_loss:.6f} | LR: {scheduler.get_last_lr()[0]:.1e}")
torch.save(
    [p.detach() for p in params_list],
    r"LOCATION TO STORE THE TRAINED WEIGHTS"
)
print("Weights saved successfully")


In [ ]:
import os
from PIL import Image
import torchvision.transforms as T
import torch

# --- Paths ---
test_folder = r"PUT YOUR TESTING FOLDER PATH HERE"  # your LR images folder
output_dir = r"PUT YOUR OUTPUT PATH HERE"
os.makedirs(output_dir, exist_ok=True)

# --- Transforms ---
to_tensor = T.ToTensor()
to_pil = T.ToPILImage()

# --- Loop through all images ---
for img_name in os.listdir(test_folder):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(test_folder, img_name)

        # Load image and convert to tensor
        lr = Image.open(img_path).convert('RGB')
        lr_t = to_tensor(lr).unsqueeze(0)  # [1,3,H,W]

        # Run model
        with torch.no_grad():
            sr_t = forward_pipeline(lr_t, params, scale=2).clamp(0,1)

        # Convert to PIL image
        sr_img = to_pil(sr_t.squeeze(0))

        # Save SR image
        output_path = os.path.join(output_dir, f"SR_{img_name}")
        sr_img.save(output_path)

        print(f"Saved SR image: {output_path}")


In [ ]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# ------------------------------------------------------------
# Load trained parameters
# ------------------------------------------------------------
saved_params = torch.load(
    r"PUT YOUR PATH TO PARAMETERS FOLDER",
    map_location="cpu"
)

params_list, params = init_params(base_channels=64, scale=2)

for p, saved_p in zip(params.values(), saved_params):
    p.data.copy_(saved_p)

# ------------------------------------------------------------
# Dataset folders
# ------------------------------------------------------------
test_folders = {
    "Set5":  r"TEST2 FOLDER PATH",
    "Set14": r"TEST1 FOLDER PATH"
}

to_tensor = T.ToTensor()
to_pil = T.ToPILImage()

# ------------------------------------------------------------
# PSNR + SSIM evaluation
# ------------------------------------------------------------
def evaluate_dataset(folder):
    psnr_scores = []
    ssim_scores = []

    for img_name in os.listdir(folder):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue
        
        # HR image
        hr = Image.open(os.path.join(folder, img_name)).convert("RGB")
        w, h = hr.size
        hr_t = to_tensor(hr).unsqueeze(0)

        # LR (bicubic)
        lr = hr.resize((w//2, h//2), Image.BICUBIC)
        lr_t = to_tensor(lr).unsqueeze(0)

        # SR output
        with torch.no_grad():
            sr_t = forward_pipeline(lr_t, params, scale=2).clamp(0, 1)

        # ---- FIX: resize SR to HR size ----
        sr_img = to_pil(sr_t.squeeze())
        sr_img = sr_img.resize((w, h), Image.BICUBIC)
        sr_np = to_tensor(sr_img).permute(1, 2, 0).numpy()

        # Convert HR → numpy
        hr_np = hr_t.squeeze().permute(1, 2, 0).numpy()

        # --- Compute PSNR & SSIM ---
        psnr_val = peak_signal_noise_ratio(hr_np, sr_np, data_range=1.0)
        ssim_val = structural_similarity(
            hr_np, sr_np,
            channel_axis=-1,
            win_size=5,
            data_range=1.0
        )

        psnr_scores.append(psnr_val)
        ssim_scores.append(ssim_val)

        print(f"{img_name} → PSNR: {psnr_val:.3f}, SSIM: {ssim_val:.4f}")

    return sum(psnr_scores)/len(psnr_scores), sum(ssim_scores)/len(ssim_scores)


# ------------------------------------------------------------
# Run for all datasets
# ------------------------------------------------------------
for set_name, folder in test_folders.items():
    avg_psnr, avg_ssim = evaluate_dataset(folder)
    print(f"\n=== {set_name} RESULTS ===")
    print(f"Average PSNR: {avg_psnr:.3f} dB")
    print(f"Average SSIM: {avg_ssim:.4f}\n")
